# RNN, CNN


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/iamtatsuki05/NLP_100/blob/NLP_100_9/NLP_100_3.ipynb)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# ID番号への変換
## 問題51で構築した学習データ中の単語にユニークなID番号を付与したい．学習データ中で最も頻出する単語に1，2番目に頻出する単語に2，……といった方法で，学習データ中で2回以上出現する単語にID番号を付与せよ．そして，与えられた単語列に対して，ID番号の列を返す関数を実装せよ．ただし，出現頻度が2回未満の単語のID番号はすべて0とせよ

In [2]:
#50,51をもう一度
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
!unzip NewsAggregatorDataset.zip
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('/content/newsCorpora.csv' , header=None , sep='\t' , names=['ID' , 'TITLE' , 'URL' , 'PUBLISHER' , 'CATEGORY' , 'STORY' , 'HOSTNAME' , 'TIMESTAMP'])

# データの抽出
df = df.loc[df['PUBLISHER'].isin(['Reuters' , 'Huffington Post' , 'Businessweek' , 'Contactmusic.com' , 'Daily Mail']) , ['TITLE' , 'CATEGORY']]

# データの分割
train, valid_test = train_test_split(df , test_size = 0.2 , shuffle = True , random_state = 42 , stratify = df['CATEGORY'])
valid, test = train_test_split(valid_test , test_size = 0.5 , shuffle = True , random_state = 42 , stratify = valid_test['CATEGORY'])
train.reset_index(drop=True , inplace = True)
valid.reset_index(drop=True , inplace = True)
test.reset_index(drop=True , inplace = True)

from collections import defaultdict
import string

d = defaultdict(int)
table = str.maketrans(string.punctuation , ' '*len(string.punctuation))#記号処理
for text in train['TITLE']:
  for word in text.translate(table).split():
    d[word] += 1
d = sorted(d.items() , key=lambda x:x[1] , reverse=True)#sort

word_id = {word: idx + 1 for idx , (word , num) in enumerate(d) if num > 1}#辞書
word_id

--2022-03-24 18:46:29--  https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29224203 (28M) [application/x-httpd-php]
Saving to: ‘NewsAggregatorDataset.zip’

NewsAggregatorDatas 100%[===================>]  27.87M  25.2MB/s    in 1.1s    

2022-03-24 18:46:31 (25.2 MB/s) - ‘NewsAggregatorDataset.zip’ saved [29224203/29224203]

Archive:  NewsAggregatorDataset.zip
  inflating: 2pageSessions.csv       
   creating: __MACOSX/
  inflating: __MACOSX/._2pageSessions.csv  
  inflating: newsCorpora.csv         
  inflating: __MACOSX/._newsCorpora.csv  
  inflating: readme.txt              
  inflating: __MACOSX/._readme.txt   


{'to': 1,
 's': 2,
 'in': 3,
 'UPDATE': 4,
 'on': 5,
 'as': 6,
 'US': 7,
 'The': 8,
 'of': 9,
 'for': 10,
 '1': 11,
 'To': 12,
 '2': 13,
 'the': 14,
 'and': 15,
 'In': 16,
 'Of': 17,
 'a': 18,
 'at': 19,
 'A': 20,
 'With': 21,
 'Is': 22,
 'For': 23,
 'And': 24,
 'with': 25,
 'after': 26,
 'New': 27,
 '3': 28,
 'Kardashian': 29,
 'China': 30,
 'up': 31,
 'On': 32,
 'Kim': 33,
 'by': 34,
 'says': 35,
 'is': 36,
 'After': 37,
 'At': 38,
 'ECB': 39,
 'STOCKS': 40,
 'From': 41,
 't': 42,
 'Fed': 43,
 'from': 44,
 '4': 45,
 'new': 46,
 'Cyrus': 47,
 'Miley': 48,
 'It': 49,
 'her': 50,
 'Wall': 51,
 'Says': 52,
 'FOREX': 53,
 'shares': 54,
 '5': 55,
 'CEO': 56,
 'Dollar': 57,
 'First': 58,
 'data': 59,
 'Google': 60,
 'West': 61,
 'over': 62,
 'she': 63,
 'About': 64,
 'St': 65,
 'May': 66,
 'S': 67,
 'Euro': 68,
 'Will': 69,
 'You': 70,
 'Ukraine': 71,
 'More': 72,
 'Chris': 73,
 'As': 74,
 'bln': 75,
 '2014': 76,
 'Kanye': 77,
 'Be': 78,
 'Up': 79,
 'Apple': 80,
 'Star': 81,
 'Over': 82,
 '

In [3]:
import torch
#テキスト修正
def tokenizer(text , word2id=word_id , unk = 0):
  table = str.maketrans(string.punctuation , ' '*len(string.punctuation))
  return [word2id.get(word , unk) for word in text.translate(table).split()]

# RNNによる予測
## ID番号で表現された単語列x=(x1,x2,…,xT)がある．ただし，Tは単語列の長さ，xt∈ℝVは単語のID番号のone-hot表記である（Vは単語の総数である）．再帰型ニューラルネットワーク（RNN: Recurrent Neural Network）を用い，単語列xからカテゴリyを予測するモデルとして，次式を実装せよ．

h→0=0,h→t=RNN−→−−(emb(xt),h→t−1),y=softmax(W(yh)h→T+b(y))
ただし，emb(x)∈ℝdwは単語埋め込み（単語のone-hot表記から単語ベクトルに変換する関数），h→t∈ℝdhは時刻tの隠れ状態ベクトル，RNN−→−−(x,h)は入力xと前時刻の隠れ状態hから次状態を計算するRNNユニット，W(yh)∈ℝL×dhは隠れ状態ベクトルからカテゴリを予測するための行列，b(y)∈ℝLはバイアス項である（dw,dh,Lはそれぞれ，単語埋め込みの次元数，隠れ状態ベクトルの次元数，ラベル数である）．RNNユニットRNN−→−−(x,h)には様々な構成が考えられるが，典型例として次式が挙げられる．

RNN−→−−(x,h)=g(W(hx)x+W(hh)h+b(h))
ただし，W(hx)∈ℝdh×dw，W(hh)∈ℝdh×dh,b(h)∈ℝdhはRNNユニットのパラメータ，gは活性化関数（例えばtanhやReLUなど）である．

なお，この問題ではパラメータの学習を行わず，ランダムに初期化されたパラメータでyを計算するだけでよい．次元数などのハイパーパラメータは，dw=300,dh=50など，適当な値に設定せよ（以降の問題でも同様である）．

In [4]:
# ，dw=300,dh=50

In [5]:
import torch
from torch import nn
torch.manual_seed(42)

class RNN(nn.Module):
  def __init__(self , vocab_size , emb_size , padding_idx , output_size , hidden_size):
    super().__init__()
    self.hidden_size = hidden_size
    self.emb = nn.Embedding(vocab_size , emb_size , padding_idx=padding_idx)
    self.rnn = nn.RNN(emb_size , hidden_size , nonlinearity='relu' , batch_first=True)
    self.fc = nn.Linear(hidden_size , output_size)
    
  def forward(self, x):
    self.batch_size = x.size()[0]
    hidden = self.init_hidden()
    emb = self.emb(x)
    out, hidden = self.rnn(emb , hidden)
    out = self.fc(out[: , -1 , :])
    return out
    
  def init_hidden(self):
    hidden = torch.zeros(1, self.batch_size , self.hidden_size)
    return hidden

In [6]:
from torch.utils.data import Dataset

class CreateDataset(Dataset):
  def __init__(self , X , y , tokenizer):
    self.X = X
    self.y = y
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.y)

  def __getitem__(self , index):
    text = self.X[index]
    inputs = self.tokenizer(text)

    return {'inputs': torch.tensor(inputs , dtype=torch.int64) , 'labels': torch.tensor(self.y[index] , dtype=torch.int64)}

In [7]:
# ラベルベクトル
category_dict = {'b': 0 , 't': 1 , 'e':2 , 'm':3}
y_train = train['CATEGORY'].map(lambda x: category_dict[x]).values
y_valid = valid['CATEGORY'].map(lambda x: category_dict[x]).values
y_test = test['CATEGORY'].map(lambda x: category_dict[x]).values

ds_train = CreateDataset(train['TITLE'] , y_train , tokenizer)
ds_valid = CreateDataset(valid['TITLE'] , y_valid , tokenizer)
ds_test = CreateDataset(test['TITLE'] , y_test , tokenizer)

vocab_size = len(set(word_id.values())) + 1
emb_size = 300
padding_idx = len(set(word_id.values()))
output_size = 4
hidden_size = 50

model = RNN(vocab_size , emb_size , padding_idx , output_size , hidden_size)

# 先頭10件の予測値取得
for num in range(10):
  X = ds_train[num]['inputs']
  print(torch.softmax(model(X.unsqueeze(0)) , dim=-1))

tensor([[0.1730, 0.4242, 0.1579, 0.2450]], grad_fn=<SoftmaxBackward0>)
tensor([[0.1941, 0.5203, 0.0718, 0.2138]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2368, 0.2368, 0.3586, 0.1677]], grad_fn=<SoftmaxBackward0>)
tensor([[0.0919, 0.3075, 0.2937, 0.3069]], grad_fn=<SoftmaxBackward0>)
tensor([[0.3009, 0.1867, 0.1556, 0.3569]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2249, 0.1537, 0.4146, 0.2068]], grad_fn=<SoftmaxBackward0>)
tensor([[0.3367, 0.4244, 0.0543, 0.1846]], grad_fn=<SoftmaxBackward0>)
tensor([[0.0801, 0.1817, 0.4518, 0.2863]], grad_fn=<SoftmaxBackward0>)
tensor([[0.1038, 0.3213, 0.1686, 0.4063]], grad_fn=<SoftmaxBackward0>)
tensor([[0.3946, 0.1217, 0.1182, 0.3655]], grad_fn=<SoftmaxBackward0>)


In [8]:
# 参考https://exture-ri.com/2021/01/12/pytorch-rnn/
# https://gotutiyan.hatenablog.com/entry/2020/09/02/200144

# 確率的勾配降下法による学習
## 確率的勾配降下法（SGD: Stochastic Gradient Descent）を用いて，問題81で構築したモデルを学習せよ．訓練データ上の損失と正解率，評価データ上の損失と正解率を表示しながらモデルを学習し，適当な基準（例えば10エポックなど）で終了させよ．

In [9]:
from torch.utils.data import DataLoader
from torch import optim

def acc(model , dataset , device =None , criterion = None):
  dataloader = DataLoader(dataset , batch_size = 1 , shuffle=False)
  loss = 0.0
  total = 0
  correct = 0
  with torch.no_grad():
    for data in dataloader:
      
      inputs = data['inputs'].to(device)#gpu
      labels = data['labels'].to(device)#gpu
      outputs = model(inputs)#計算
      if criterion != None:
        loss += criterion(outputs, labels).item()#loss
        #正解率
      pred = torch.argmax(outputs , dim=-1)
      total += len(inputs)
      correct += (pred == labels).sum().item()
      
  return loss / len(dataset) , correct / total
  

def train_model(dataset_train , dataset_valid , batch_size , model , criterion , op , num_epochs , collate_fn = None , device = None):
  model.to(device)
  dataloader_train = DataLoader(dataset_train , batch_size = batch_size , shuffle=True , collate_fn = collate_fn)
  dataloader_valid = DataLoader(dataset_valid , batch_size = 1, shuffle = False)
  scheduler = optim.lr_scheduler.CosineAnnealingLR(op , num_epochs , eta_min=1e-3, last_epoch = -1)
  loss_train_list = []
  loss_valid_list = []

  for epoch in range(num_epochs):
    model.train()
    for data in dataloader_train:
      op.zero_grad()#初期化
      inputs = data['inputs'].to(device)#gpu
      labels = data['labels'].to(device)#gpu
      #計算
      outputs = model.forward(inputs)
      loss = criterion(outputs, labels)
      loss.backward()#調節
      op.step()#更新
    
    model.eval()

    # 損失と正解率の算出
    loss_train , acc_train = acc(model , dataset_train , device , criterion = criterion)
    loss_valid , acc_valid = acc(model , dataset_valid , device , criterion = criterion)
    loss_train_list.append([loss_train , acc_train])
    loss_valid_list.append([loss_valid , acc_valid])

    #パラメータ保存
    torch.save({'epoch': epoch , 'model_state_dict': model.state_dict() , 'optimizer_state_dict': op.state_dict()} , f'checkpoint{epoch + 1}.pt')

    #正答率誤算などの算出
    print(f'epoch: {epoch + 1}, loss_train: {loss_train:.4f}, acc_train: {acc_train:.4f}, loss_valid: {loss_valid:.4f}, acc_valid: {acc_valid:.4f}') 
      
    scheduler.step()

  return {'train': loss_train, 'valid': loss_valid}

In [10]:
vocab_size = len(set(word_id.values())) + 1
emb_size = 300
padding_idx = len(set(word_id.values()))
output_size = 4
hidden_size = 50
learning_rate = 1e-3
batch_size = 1
num_epochs = 3#testのため回数少なめ

model = RNN(vocab_size , emb_size , padding_idx , output_size , hidden_size)
CE = nn.CrossEntropyLoss()
op = torch.optim.SGD(model.parameters() , lr = learning_rate)

log = train_model(ds_train , ds_valid , batch_size , model , CE , op , num_epochs)

epoch: 1, loss_train: 1.0963, acc_train: 0.5208, loss_valid: 1.1103, acc_valid: 0.5217
epoch: 2, loss_train: 0.9395, acc_train: 0.6363, loss_valid: 0.9795, acc_valid: 0.6079
epoch: 3, loss_train: 0.7186, acc_train: 0.7396, loss_valid: 0.7964, acc_valid: 0.7189


In [11]:
# https://note.nkmk.me/python-pytorch-device-to-cuda-cpu/
# https://runebook.dev/ja/docs/pytorch/generated/torch.nn.bcewithlogitsloss
# https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html
# https://tips-memo.com/python-diff-bce

# ミニバッチ化・GPU上での学習
## 問題82のコードを改変し，B事例ごとに損失・勾配を計算して学習を行えるようにせよ（Bの値は適当に選べ）．また，GPU上で学習を実行せよ．

In [12]:
class Padsequence():
  def __init__(self, padding_idx):
    self.padding_idx = padding_idx

  def __call__(self, batch):
    sorted_batch = sorted(batch, key=lambda x: x['inputs'].shape[0], reverse=True)
    sequences = [x['inputs'] for x in sorted_batch]
    sequences_padded = torch.nn.utils.rnn.pad_sequence(sequences, batch_first=True, padding_value=self.padding_idx)
    labels = torch.LongTensor([x['labels'] for x in sorted_batch])

    return {'inputs': sequences_padded, 'labels': labels}

In [13]:
vocab_size = len(set(word_id.values())) + 1
emb_size = 300
padding_idx = len(set(word_id.values()))
output_size = 4
hidden_size = 50
learning_rate = 1e-3
batch_size = 50
num_epochs = 3#testのため回数少なめ

model = RNN(vocab_size , emb_size , padding_idx , output_size , hidden_size)
CE = nn.CrossEntropyLoss()
op = torch.optim.SGD(model.parameters() , lr = learning_rate)
# device = torch.device('cuda')#gpu
device = torch.device('cpu')#gpuに制限がかかったのでcpu

log = train_model(ds_train , ds_valid , batch_size , model , CE , op , num_epochs , collate_fn = Padsequence(padding_idx) , device = device)

epoch: 1, loss_train: 1.4867, acc_train: 0.2546, loss_valid: 1.4858, acc_valid: 0.2579
epoch: 2, loss_train: 1.3978, acc_train: 0.3071, loss_valid: 1.3949, acc_valid: 0.3073
epoch: 3, loss_train: 1.3337, acc_train: 0.3513, loss_valid: 1.3303, acc_valid: 0.3501


In [14]:
# 参考https://atmarkit.itmedia.co.jp/ait/articles/2008/28/news030.html

# 単語ベクトルの導入
## 事前学習済みの単語ベクトル（例えば，Google Newsデータセット（約1,000億単語）での学習済み単語ベクトル）で単語埋め込みemb(x)を初期化し，学習せよ．

In [15]:
# ダウンロード制限がかかっているのでprthを指定する
# ! pip install --upgrade gdown
# import gdown
# gdown.download('https://drive.google.com/u/0/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM&export=download', './GoogleNews-vectors-negative300.bin.gz', quiet=False)
# #model
# from gensim.models import KeyedVectors

# # 学習済みモデルのロード
# model = KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin.gz' , binary=True)

In [16]:
# 直接pathを指定する場合
from gensim.models import KeyedVectors
model_kv = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Tutorial/NLP_100/GoogleNews-vectors-negative300.bin.gz' , binary=True)

In [17]:
import numpy as np

vocab_size = len(set(word_id.values())) + 1
emb_size = 300
weights = np.zeros((vocab_size , emb_size))
words_in_pretrained = 0
for idx , word in enumerate(word_id.keys()):
  try:
    weights[idx] = model_kv[word]
    words_in_pretrained += 1
  except KeyError:
    weights[idx] = np.random.normal(    loc   = 0 , scale = 1 , size=(emb_size ,))#正規化
weights = torch.from_numpy(weights.astype((np.float32)))#torch

In [18]:
class RNN(nn.Module):
  def __init__(self , vocab_size , emb_size , padding_idx , output_size , hidden_size , num_layers , emb_weights = None , bidirectional = False):
    super().__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.num_directions = bidirectional + 1

    if emb_weights != None:
      self.emb = nn.Embedding.from_pretrained(emb_weights, padding_idx=padding_idx)
    else:
      self.emb = nn.Embedding(vocab_size, emb_size, padding_idx=padding_idx)

    self.rnn = nn.RNN(emb_size , hidden_size , num_layers , nonlinearity='relu' , bidirectional = bidirectional , batch_first = True)
    self.fc = nn.Linear(hidden_size * self.num_directions , output_size)
    
  def forward(self, x):
    self.batch_size = x.size()[0]
    hidden = self.init_hidden()
    emb = self.emb(x)
    out, hidden = self.rnn(emb, hidden)
    out = self.fc(out[: , -1 , :])
    return out
    
  def init_hidden(self):
    hidden = torch.zeros(self.num_layers * self.num_directions , self.batch_size , self.hidden_size)
    return hidden

In [19]:
vocab_size = len(set(word_id.values())) + 1
emb_size = 300
padding_idx = len(set(word_id.values()))
output_size = 4
hidden_size = 50
num_layers = 1
learning_rate = 1e-3
batch_size = 50
num_epochs = 3#testのため回数少なめ

model = RNN(vocab_size , emb_size , padding_idx , output_size , hidden_size , num_layers , emb_weights = weights)
CE = nn.CrossEntropyLoss()
op = torch.optim.SGD(model.parameters() , lr = learning_rate)
# device = torch.device('cuda')#gpu
device = torch.device('cpu')#gpuに制限がかかったのでcpu

log = train_model(ds_train , ds_valid , batch_size , model , CE , op , num_epochs , collate_fn = Padsequence(padding_idx) , device = device)

epoch: 1, loss_train: 1.3848, acc_train: 0.2868, loss_valid: 1.3950, acc_valid: 0.2609
epoch: 2, loss_train: 1.3442, acc_train: 0.3618, loss_valid: 1.3514, acc_valid: 0.3358
epoch: 3, loss_train: 1.3117, acc_train: 0.3794, loss_valid: 1.3168, acc_valid: 0.3681


In [20]:
# 参考https://www.sejuku.net/blog/73026
# https://note.nkmk.me/python-numpy-dtype-astype/

# 双方向RNN・多層化
## 
順方向と逆方向のRNNの両方を用いて入力テキストをエンコードし，モデルを学習せよ．

h⃖ T+1=0,h⃖ t=RNN←−−−(emb(xt),h⃖ t+1),y=softmax(W(yh)[h→T;h⃖ 1]+b(y))
ただし，h→t∈ℝdh,h⃖ t∈ℝdhはそれぞれ，順方向および逆方向のRNNで求めた時刻tの隠れ状態ベクトル，RNN←−−−(x,h)は入力xと次時刻の隠れ状態hから前状態を計算するRNNユニット，W(yh)∈ℝL×2dhは隠れ状態ベクトルからカテゴリを予測するための行列，b(y)∈ℝLはバイアス項である．また，[a;b]はベクトルaとbの連結を表す。

さらに，双方向RNNを多層化して実験せよ．

In [21]:
vocab_size = len(set(word_id.values())) + 1
emb_size = 300
padding_idx = len(set(word_id.values()))
output_size = 4
hidden_size = 50
num_layers = 5#ここで多層化
learning_rate = 1e-3
batch_size = 50
num_epochs = 3#testのため回数少なめ

model = RNN(vocab_size , emb_size , padding_idx , output_size , hidden_size , num_layers , emb_weights = weights , bidirectional = True)#bidirectional = Trueで双方向
CE = nn.CrossEntropyLoss()
op = optim.SGD(model.parameters() , lr = learning_rate)
# device = torch.device('cuda')#gpu
device = torch.device('cpu')#gpuに制限がかかったのでcpu

log = train_model(ds_train , ds_valid , batch_size , model , CE , op , num_epochs , collate_fn = Padsequence(padding_idx) , device = device)

epoch: 1, loss_train: 1.3380, acc_train: 0.3966, loss_valid: 1.3386, acc_valid: 0.3951
epoch: 2, loss_train: 1.2850, acc_train: 0.3982, loss_valid: 1.2854, acc_valid: 0.3996
epoch: 3, loss_train: 1.2421, acc_train: 0.3978, loss_valid: 1.2424, acc_valid: 0.3988


In [22]:
# 参考https://qiita.com/m__k/items/78a5125d719951ca98d3
# https://axa.biopapyrus.jp/deep-learning/rnn/brnn.html
# https://deepage.net/deep_learning/2017/05/23/recurrent-neural-networks.html
# https://teratail.com/questions/185713
# https://qiita.com/tetsuro_skiing/items/87c0c37cefd7b601f974

#  畳み込みニューラルネットワーク (CNN)
## ID番号で表現された単語列x=(x1,x2,…,xT)がある．ただし，Tは単語列の長さ，xt∈ℝVは単語のID番号のone-hot表記である（Vは単語の総数である）．畳み込みニューラルネットワーク（CNN: Convolutional Neural Network）を用い，単語列xからカテゴリyを予測するモデルを実装せよ．

ただし，畳み込みニューラルネットワークの構成は以下の通りとする．

単語埋め込みの次元数: dw
畳み込みのフィルターのサイズ: 3 トークン
畳み込みのストライド: 1 トークン
畳み込みのパディング: あり
畳み込み演算後の各時刻のベクトルの次元数: dh
畳み込み演算後に最大値プーリング（max pooling）を適用し，入力文をdh次元の隠れベクトルで表現
すなわち，時刻tの特徴ベクトルpt∈ℝdhは次式で表される．

pt=g(W(px)[emb(xt−1);emb(xt);emb(xt+1)]+b(p))
ただし，W(px)∈ℝdh×3dw,b(p)∈ℝdhはCNNのパラメータ，gは活性化関数（例えばtanhやReLUなど），[a;b;c]はベクトルa,b,cの連結である．なお，行列W(px)の列数が3dwになるのは，3個のトークンの単語埋め込みを連結したものに対して，線形変換を行うためである．

最大値プーリングでは，特徴ベクトルの次元毎に全時刻における最大値を取り，入力文書の特徴ベクトルc∈ℝdhを求める．c[i]でベクトルcのi番目の次元の値を表すことにすると，最大値プーリングは次式で表される．

c[i]=max1≤t≤Tpt[i]
最後に，入力文書の特徴ベクトルcに行列W(yc)∈ℝL×dhとバイアス項b(y)∈ℝLによる線形変換とソフトマックス関数を適用し，カテゴリyを予測する．

y=softmax(W(yc)c+b(y))
なお，この問題ではモデルの学習を行わず，ランダムに初期化された重み行列でyを計算するだけでよい．



In [23]:
# 単語埋め込みの次元数: dw
# 畳み込みのフィルターのサイズ: 3 トークン
# 畳み込みのストライド: 1 トークン
# 畳み込みのパディング: あり
# 畳み込み演算後の各時刻のベクトルの次元数: dh

In [24]:
from torch.nn import functional as F

class CNN(nn.Module):
  def __init__(self , vocab_size , emb_size , padding_idx , output_size , out_channels , kernel_heights , stride , padding , emb_weights = None):
    super().__init__()
    if emb_weights != None:
      self.emb = nn.Embedding.from_pretrained(emb_weights , padding_idx = padding_idx)
    else:
      self.emb = nn.Embedding(vocab_size , emb_size , padding_idx = padding_idx)
    self.conv = nn.Conv2d(1 , out_channels , (kernel_heights , emb_size) , stride , (padding , 0))
    self.drop = nn.Dropout(0.3)
    self.fc = nn.Linear(out_channels, output_size)
    
  def forward(self, x):
    emb = self.emb(x).unsqueeze(1)
    conv = self.conv(emb)
    act = F.relu(conv.squeeze(3))
    max_pool = F.max_pool1d(act, act.size()[2])
    out = self.fc(self.drop(max_pool.squeeze(2)))
    return out

In [25]:
vocab_size = len(set(word_id.values())) + 1
emb_size = 300
padding_idx = len(set(word_id.values()))
output_size = 4
#CNNのパラメータ
out_channels =100
kernel_heights = 3
stride = 1
padding = 1

model = CNN(vocab_size , emb_size , padding_idx , output_size , out_channels , kernel_heights , stride , padding , emb_weights = weights)

In [26]:
for num in range(10):
  X = ds_train[num]['inputs']
  print(torch.softmax(model(X.unsqueeze(0)) , dim=-1))

tensor([[0.2496, 0.2184, 0.2293, 0.3027]], grad_fn=<SoftmaxBackward0>)
tensor([[0.3292, 0.2204, 0.1507, 0.2997]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2730, 0.2025, 0.2404, 0.2841]], grad_fn=<SoftmaxBackward0>)
tensor([[0.3409, 0.1346, 0.1916, 0.3329]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2808, 0.2080, 0.2482, 0.2629]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2545, 0.2165, 0.2067, 0.3224]], grad_fn=<SoftmaxBackward0>)
tensor([[0.3830, 0.1757, 0.1111, 0.3301]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2838, 0.2415, 0.1816, 0.2930]], grad_fn=<SoftmaxBackward0>)
tensor([[0.3962, 0.2543, 0.1292, 0.2202]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2869, 0.2070, 0.2128, 0.2933]], grad_fn=<SoftmaxBackward0>)


In [27]:
# 参考https://qiita.com/shu_marubo/items/70b20c3a6c172aaeb8de
# https://qiita.com/mathlive/items/8e1f9a8467fff8dfd03c
# https://exture-ri.com/2021/01/11/pytorch-cnn/
# https://qiita.com/m__k/items/6c39cfe7dfa99102fa8e
# https://kento1109.hatenablog.com/entry/2019/09/30/115139

# 確率的勾配降下法によるCNNの学習
## 確率的勾配降下法（SGD: Stochastic Gradient Descent）を用いて，問題86で構築したモデルを学習せよ．訓練データ上の損失と正解率，評価データ上の損失と正解率を表示しながらモデルを学習し，適当な基準（例えば10エポックなど）で終了させよ．

In [28]:
vocab_size = len(set(word_id.values())) + 1
emb_size = 300
padding_idx = len(set(word_id.values()))
output_size = 4
#CNNのパラメータ
out_channels =100
kernel_heights = 3
stride = 1
padding = 1
learning_rate = 1e-3
batch_size = 50
num_epochs = 3#testのため回数少なめ

model = CNN(vocab_size , emb_size , padding_idx , output_size , out_channels , kernel_heights , stride , padding , emb_weights = weights)
CE = nn.CrossEntropyLoss()
op = optim.SGD(model.parameters() , lr = learning_rate)
# device = torch.device('cuda')#gpu
device = torch.device('cpu')#gpuに制限がかかったのでcpu

# モデルの学習
log = train_model(ds_train , ds_valid , batch_size , model , CE , op , num_epochs , collate_fn = Padsequence(padding_idx) , device = device)

epoch: 1, loss_train: 1.2421, acc_train: 0.4727, loss_valid: 1.2253, acc_valid: 0.4663
epoch: 2, loss_train: 1.2081, acc_train: 0.5047, loss_valid: 1.1996, acc_valid: 0.4805
epoch: 3, loss_train: 1.1859, acc_train: 0.5233, loss_valid: 1.1816, acc_valid: 0.5082


# パラメータチューニング
## 問題85や問題87のコードを改変し，ニューラルネットワークの形状やハイパーパラメータを調整しながら，高性能なカテゴリ分類器を構築せよ．

In [29]:
class RNN(nn.Module):
  def __init__(self , vocab_size , emb_size , padding_idx , output_size , hidden_size , num_layers , emb_weights = None , bidirectional = False):
    super().__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.num_directions = bidirectional + 1

    if emb_weights != None:
      self.emb = nn.Embedding.from_pretrained(emb_weights, padding_idx=padding_idx)
    else:
      self.emb = nn.Embedding(vocab_size, emb_size, padding_idx=padding_idx)

    self.rnn = nn.RNN(emb_size , hidden_size , num_layers , nonlinearity='relu' , bidirectional = bidirectional , batch_first = True)
    self.fc = nn.Linear(hidden_size * self.num_directions , output_size)
    
  def forward(self, x):
    self.batch_size = x.size()[0]
    hidden = self.init_hidden()
    emb = self.emb(x)
    out, hidden = self.rnn(emb, hidden)
    out = self.fc(out[: , -1 , :])
    return out
    
  def init_hidden(self):
    hidden = torch.zeros(self.num_layers * self.num_directions , self.batch_size , self.hidden_size)
    return hidden

In [30]:
vocab_size = len(set(word_id.values())) + 1
emb_size = 300
padding_idx = len(set(word_id.values()))
output_size = 4
hidden_size = 50
num_layers = 10#多い法が良い？
learning_rate = 1e-3#小さい方がいい？
batch_size = 128#適度に増やす
num_epochs = 50#多めに

model = RNN(vocab_size , emb_size , padding_idx , output_size , hidden_size , num_layers , emb_weights = weights , bidirectional = True)
CE = nn.CrossEntropyLoss()
op = optim.SGD(model.parameters() , lr = learning_rate)
# device = torch.device('cuda')#gpu
device = torch.device('cpu')#gpuに制限がかかったのでcpu

log = train_model(ds_train , ds_valid , batch_size , model , CE , op , num_epochs , collate_fn = Padsequence(padding_idx) , device = device)

epoch: 1, loss_train: 1.3852, acc_train: 0.3953, loss_valid: 1.3851, acc_valid: 0.3951
epoch: 2, loss_train: 1.3603, acc_train: 0.3957, loss_valid: 1.3603, acc_valid: 0.3958
epoch: 3, loss_train: 1.3375, acc_train: 0.3957, loss_valid: 1.3375, acc_valid: 0.3958
epoch: 4, loss_train: 1.3160, acc_train: 0.3957, loss_valid: 1.3160, acc_valid: 0.3958
epoch: 5, loss_train: 1.2956, acc_train: 0.3957, loss_valid: 1.2956, acc_valid: 0.3958
epoch: 6, loss_train: 1.2769, acc_train: 0.3957, loss_valid: 1.2770, acc_valid: 0.3958
epoch: 7, loss_train: 1.2598, acc_train: 0.3957, loss_valid: 1.2599, acc_valid: 0.3958
epoch: 8, loss_train: 1.2441, acc_train: 0.3957, loss_valid: 1.2442, acc_valid: 0.3958
epoch: 9, loss_train: 1.2298, acc_train: 0.3994, loss_valid: 1.2299, acc_valid: 0.4063
epoch: 10, loss_train: 1.2169, acc_train: 0.4224, loss_valid: 1.2171, acc_valid: 0.4220
epoch: 11, loss_train: 1.2058, acc_train: 0.4223, loss_valid: 1.2060, acc_valid: 0.4220
epoch: 12, loss_train: 1.1964, acc_train:

In [31]:
# 参考https://qiita.com/nyanko-box/items/a6f50e28383a5bd0a432
# https://cpp-learning.com/optuna-pytorch/
# https://qiita.com/Yushi1958/items/cd22ade638f7e292e520
# https://dreamer-uma.com/pytorch-optuna-hyperparameter-tuning/
# http://maruo51.com/2020/08/07/optuna_pytorch/
# https://ichi.pro/optuna-o-shiyoshita-pytorch-haipa-parame-ta-no-chosei-4883072668892

# 事前学習済み言語モデルからの転移学習
## 事前学習済み言語モデル（例えばBERTなど）を出発点として，ニュース記事見出しをカテゴリに分類するモデルを構築せよ．

In [32]:
# !pip install -q transformers
# from transformers import BertTokenizer, BertModel
# from torch import cuda

In [33]:
# https://note.nkmk.me/python-pytorch-device-to-cuda-cpu/
# https://qiita.com/yamaru/items/63a342c844cff056a549
# https://qiita.com/m__k/items/e312ddcf9a3d0ea64d72
# https://scrapbox.io/miyamonz/pytorch,_transformers%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%9FBERT%E3%81%AEfine-tuning%E3%81%AE%E6%96%B9%E6%B3%95